In [1]:
from covid import utils
import numpy as np
import pandas as pd

In [2]:
individuals_path =  "/Users/sandernordeide/Epidemiologic-Vaccine-Strategies/GA_20210507130042/individuals/individuals_0.pkl"
score_path =        "/Users/sandernordeide/Epidemiologic-Vaccine-Strategies/GA_20210507130042/final_scores/final_score_0.pkl"
overview_path =     "/Users/sandernordeide/Epidemiologic-Vaccine-Strategies/GA_20210507130042/generation_overview.csv"

final_score = utils.read_pickle(score_path)
individuals = utils.read_pickle(individuals_path)
overview = pd.read_csv(overview_path)

In [3]:
final_score

{140281340491808: [902.0,
  889.0,
  920.0,
  907.0,
  832.0,
  830.0,
  857.0,
  844.0,
  954.0,
  806.0,
  792.0,
  896.0,
  884.0,
  848.0,
  908.0,
  841.0,
  949.0,
  979.0,
  936.0,
  831.0],
 140281340490704: [167.0,
  139.0,
  173.0,
  152.0,
  144.0,
  146.0,
  166.0,
  150.0,
  160.0,
  115.0,
  145.0,
  170.0,
  144.0,
  139.0,
  145.0,
  150.0,
  195.0,
  216.0,
  209.0,
  144.0],
 140281340491568: [168.0,
  138.0,
  174.0,
  151.0,
  144.0,
  146.0,
  166.0,
  150.0,
  160.0,
  115.0,
  147.0,
  172.0,
  145.0,
  142.0,
  145.0,
  150.0,
  195.0,
  215.0,
  208.0,
  144.0],
 140281340492432: [46.0,
  29.0,
  65.0,
  33.0,
  45.0,
  54.0,
  49.0,
  47.0,
  42.0,
  16.0,
  46.0,
  50.0,
  43.0,
  47.0,
  42.0,
  29.0,
  71.0,
  69.0,
  64.0,
  48.0],
 140281340491664: [168.0,
  140.0,
  173.0,
  151.0,
  144.0,
  146.0,
  165.0,
  150.0,
  160.0,
  115.0,
  146.0,
  171.0,
  145.0,
  139.0,
  145.0,
  150.0,
  195.0,
  215.0,
  208.0,
  144.0],
 140281340490272: [77.0,
  56.

In [4]:
for ID in final_score.keys():
    print(np.mean(final_score[ID]))

880.25
158.45
158.75
46.75
158.5
73.35
73.3
83.05
90.05
60.8


In [5]:
for i in individuals:
    print(i.ID)

140281340492432
140281340493632
140281340490176
140281340490272
140281340490992
140281340491472
140281340490704
140281340491664
140281340491568
140281340491808


In [6]:
overview.head(20)

,generation,individual,mean_score
0,0,140281340492432,46.75
1,0,140281340493632,60.80
2,0,140281340490176,73.30
3,0,140281340490272,73.35
4,0,140281340490992,83.05
5,0,140281340491472,90.05
6,0,140281340490704,158.45
7,0,140281340491664,158.50
8,0,140281340491568,158.75
9,0,140281340491808,880.25
